## Lectura PQR

In [1]:
import MDAnalysis as mda
from MDAnalysis import transformations
import numpy as np

zika_origin = mda.Universe('pqr/ZIKV_6CO8_aa_charge_vdw_addspace.pqr')
zika = mda.Universe('pqr/ZIKV_6CO8_aa_charge_vdw_addspace.pqr')

In [2]:
zika_origin.atoms.positions, zika_origin.atoms.positions.shape

(array([[-158.378,  -64.7  , -129.804],
        [-159.515,  -65.576, -129.445],
        [-159.178,  -66.976, -129.96 ],
        ...,
        [ -91.22 ,  146.41 ,   73.082],
        [ -91.007,  149.585,   75.407],
        [ -90.401,  146.744,   72.613]], dtype=float32),
 (1576628, 3))

In [3]:
ts = zika.trajectory.ts
angle = 180 # Angle in degrees
ag = zika.atoms
d = [0,0,1] # direction of a custom axis of rotation from the provided point
rotated = transformations.rotate.rotateby(angle, direction=d, ag=ag)(ts)

In [4]:
rotated.positions, rotated.positions.shape

(array([[ 158.38007,   64.70003, -129.804  ],
        [ 159.51706,   65.57603, -129.445  ],
        [ 159.18005,   66.97603, -129.96   ],
        ...,
        [  91.22207, -146.40997,   73.082  ],
        [  91.00907, -149.58498,   75.407  ],
        [  90.40307, -146.74397,   72.613  ]], dtype=float32),
 (1576628, 3))

In [5]:
# Checkeo de seguridad
np.where(abs((zika_origin.atoms.positions[:,0] - -rotated.positions[:,0]))>1e-2)

(array([], dtype=int64),)

In [11]:
# Guardar archivo formato pqr!!! 
rotation_axis = np.argmax(d)
direction_to_string = lambda x : 'x-axis' if x == 0 else 'y-axis' if x == 1 else 'z-axis'
zika.atoms.write('zika_rotation_angle%2d_%s.pqr'%(angle, direction_to_string(rotation_axis)))

In [48]:
zika.atoms.ids, zika.atoms.names, zika.atoms.resnames, zika.atoms.segids, zika.atoms.resnums, zika.atoms.charges, zika.atoms.radii

(array([    1,     2,     3, ..., 76626, 76627, 76628]),
 array(['N', 'CA', 'C', ..., 'OG', 'OXT', 'HG'], dtype=object),
 array(['ILE', 'ILE', 'ILE', ..., 'SER', 'SER', 'SER'], dtype=object),
 array(['A', 'A', 'A', ..., 'F', 'F', 'F'], dtype=object),
 array([ 1,  1,  1, ..., 75, 75, 75]),
 array([ 0.031     ,  0.026     ,  0.61199999, ..., -0.65100002,
        -0.81300002,  0.447     ]),
 array([1.824     , 1.90799999, 1.90799999, ..., 1.72099996, 1.66100001,
        0.        ]))

In [60]:
# open a file and write rotation coordinates 
# no poner tantos .write en el loop
with open('zika_rotation_angle%2d_%s.pqr' %(angle, direction_to_string(rotation_axis)), 'w') as file:
    for j in range(zika.atoms.n_atoms):
        file.write('ATOM  {:5d} {:4} {:3} {}   {:3d}     {:.3f}  {:.3f} {:.3f} {:.3f} {:.3f} \n'.format(zika.atoms.ids[j], \
        zika.atoms.names[j], zika.atoms.resnames[j], zika.atoms.segids[j], zika.atoms.resnums[j], rotated.positions[j,0], \
        rotated.positions[j,1], rotated.positions[j,2], zika.atoms.charges[j], zika.atoms.radii[j]))

KeyboardInterrupt: 